# WARNING: This code doesn't work with the torch version that modAL needs, so to be able to run this script you need to do the following:
- Run "pip install --upgrade scikit-learn skorch" to update skorch
- Run this script
- Run "pip install skorch==0.9.0" to downgrade skorch to the newest version that is compatible with ModAL

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install skorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.9/228.9 kB 7.3 MB/s eta 0:00:00


In [3]:
!pip install git+https://github.com/modAL-python/modAL.git

  Cloning https://github.com/modAL-python/modAL.git to /tmp/pip-req-build-xi4z7z1i
  Running command git clone --filter=blob:none --quiet https://github.com/modAL-python/modAL.git /tmp/pip-req-build-xi4z7z1i
  Resolved https://github.com/modAL-python/modAL.git to commit bba6f6fd00dbb862b1e09259b78caf6cffa2e755
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.8/125.8 kB 4.0 MB/s eta 0:00:00
  Created wheel for modAL-python: filename=modAL_python-0.4.2-py3-none-any.whl size=32647 sha256=f2d8935b0634796fa213ab6b92331d93e9757cc5c8fd44ededd250f12419c79f
  Stored in directory: /tmp/pip-ephem-wheel-cache-g3vycyxv/wheels/87/f9/71/88b176e3d0684b68b377241dc3da25e38bd3133c2dd23fbba4
Successfully built modAL-python
  Attempting uninstall: skorch
    Found existing installation: skorch 1.1.0
    Uninstalling skorch-1.1.0:
      Successfully uninstalled skorch-1.1.0


In [4]:
!pip install skorch==1.1.0

  Using cached skorch-1.1.0-py3-none-any.whl.metadata (11 kB)
Using cached skorch-1.1.0-py3-none-any.whl (228 kB)
  Attempting uninstall: skorch
    Found existing installation: skorch 0.9.0
    Uninstalling skorch-0.9.0:
      Successfully uninstalled skorch-0.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
modal-python 0.4.2 requires skorch==0.9.0, but you have skorch 1.1.0 which is incompatible.


In [5]:
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, StratifiedKFold
from cnn_model import CNN
from skorch import NeuralNetClassifier
from sklearn.metrics import make_scorer, accuracy_score
import numpy as np
import torch
from utils import load_CIFAR, save_file
import warnings
from sklearn.exceptions import ConvergenceWarning
import time
import os, pickle
from skorch.callbacks import GradientNormClipping


warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [6]:
RANDOM_SEED = 42

Define the parameter grid: This is done in a vertain way because certain solvers are only compatible with certain regularization methods! E.g. LBFGS is only compatible without regularization or L2.

In [7]:
param_grid = {
    # CNN-specific parameters
    'module__input_size': [32],
    'module__num_channels': [3],
    'module__l1_channels': [32, 64],
    'module__l1_kernel_size': [3, 5],
    'module__l1_padding': [1, 'same'],
    'module__l1_stride': [1, 2],
    'module__l2_channels': [64, 128],
    'module__l2_kernel_size': [3, 5],
    'module__l2_max_pool_kernel_size': [3, 5],
    'module__l2_padding': [1, 'same'],
    'module__l2_stride': [1, 2],
    'module__l2_dropout': [0.2, 0.25, 0.3],
    'module__l3_dropout': [0.2, 0.25, 0.3],
    'module__l4_input': [512, 1024, 2048],
    'module__l4_dropout': [0.2, 0.25, 0.3],
    'module__l5_input': [1024, 2048],
    'module__output_size': [10],
    # Training-specific parameters
    'lr': [5e-4, 1e-4, 1e-3],
    'optimizer__weight_decay': [1e-4, 1e-3, 5e-3],
    'max_epochs': [125, 150, 175],
    'batch_size': [32, 64, 128, 256]
}


In [8]:
# Cross-validation settings
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Create the Logistic Regression model
gradient_clipping = GradientNormClipping(gradient_clip_value=1.0)
cnn_basic = NeuralNetClassifier(
    module=CNN,
    criterion=torch.nn.CrossEntropyLoss,
    optimizer=torch.optim.Adam,
    train_split=None,
    verbose=0,
    device="cuda" if torch.cuda.is_available() else "cpu",
    callbacks=[gradient_clipping]
)

# Setup GridSearchCV
random_search = RandomizedSearchCV(
    estimator=cnn_basic,
    param_distributions=param_grid,
    n_iter=10,  # Number of configurations to try
    scoring='accuracy',
    cv=cv,
    random_state=RANDOM_SEED,
    verbose=3
)

Loading MNIST

In [9]:
X_train, y_train, _, _, _, _, _, _ = load_CIFAR(random_seed=RANDOM_SEED)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [10]:
# Perform grid search
start = time.time()
random_search.fit(X_train, y_train)

# Output the best parameters and corresponding score
print("____________")
print("Best Parameters:", random_search.best_params_)
print("Best Cross-Validation Accuracy:", random_search.best_score_)
print("time:", time.time()-start)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END batch_size=128, lr=0.001, max_epochs=150, module__input_size=32, module__l1_channels=64, module__l1_kernel_size=5, module__l1_padding=1, module__l1_stride=2, module__l2_channels=64, module__l2_dropout=0.2, module__l2_kernel_size=3, module__l2_max_pool_kernel_size=3, module__l2_padding=same, module__l2_stride=1, module__l3_dropout=0.25, module__l4_dropout=0.25, module__l4_input=2048, module__l5_input=2048, module__num_channels=3, module__output_size=10, optimizer__weight_decay=0.001;, score=0.733 total time= 3.8min
[CV 2/5] END batch_size=128, lr=0.001, max_epochs=150, module__input_size=32, module__l1_channels=64, module__l1_kernel_size=5, module__l1_padding=1, module__l1_stride=2, module__l2_channels=64, module__l2_dropout=0.2, module__l2_kernel_size=3, module__l2_max_pool_kernel_size=3, module__l2_padding=same, module__l2_stride=1, module__l3_dropout=0.25, module__l4_dropout=0.25, module__l4_input=2048, module_

In [11]:
del(X_train)
del(y_train)
_, _, X_test, y_test, _, _, _, _ = load_CIFAR(random_seed=RANDOM_SEED)
# Evaluate the best model on the test set
best_model = random_search.best_estimator_
test_accuracy = best_model.score(X_test, y_test)
print("Test Accuracy with Best Model:", test_accuracy)

Test Accuracy with Best Model: 0.7447


Save parameters

In [12]:
best_params = random_search.best_params_
print(best_params)
# transform them to the format of dict that I use in the shallow_classifier experiments
model_parameters={
    # CNN-specific parameters
    'input_size': best_params['module__input_size'],
    'num_channels': best_params['module__num_channels'],
    'l1_channels': best_params['module__l1_channels'],
    'l1_kernel_size': best_params['module__l1_kernel_size'],
    'l1_padding': best_params['module__l1_padding'],
    'l1_stride': best_params['module__l1_stride'],
    'l2_channels': best_params['module__l2_channels'],
    'l2_kernel_size': best_params['module__l2_kernel_size'],
    'l2_max_pool_kernel_size': best_params['module__l2_max_pool_kernel_size'],
    'l2_padding': best_params['module__l2_padding'],
    'l2_stride': best_params['module__l2_stride'],
    'l2_dropout': best_params['module__l2_dropout'],
    'l3_dropout': best_params['module__l3_dropout'],
    'l4_input': best_params['module__l4_input'],
    'l4_dropout': best_params['module__l4_dropout'],
    'l5_input': best_params['module__l5_input'],
    'output_size': best_params['module__output_size'],
    # Training-specific parameters
    'lr': best_params['lr'],
    'weight_decay': best_params['optimizer__weight_decay'],
    'max_epochs': best_params['max_epochs'],
    'batch_size': best_params['batch_size']
}
#save_file("deep_classifier_parameters.pkl", model_parameters)
base_path = f"/content/drive/MyDrive/"
os.makedirs(base_path, exist_ok=True)
params_path = os.path.join(base_path, "deep_classifier_parameters.pkl")
with open(params_path, "wb") as f:
    pickle.dump(model_parameters, f)
print(f"Best parameters saved to {params_path}")

{'optimizer__weight_decay': 0.0001, 'module__output_size': 10, 'module__num_channels': 3, 'module__l5_input': 1024, 'module__l4_input': 2048, 'module__l4_dropout': 0.3, 'module__l3_dropout': 0.3, 'module__l2_stride': 1, 'module__l2_padding': 1, 'module__l2_max_pool_kernel_size': 5, 'module__l2_kernel_size': 5, 'module__l2_dropout': 0.2, 'module__l2_channels': 64, 'module__l1_stride': 1, 'module__l1_padding': 1, 'module__l1_kernel_size': 5, 'module__l1_channels': 64, 'module__input_size': 32, 'max_epochs': 175, 'lr': 0.0005, 'batch_size': 256}
Best parameters saved to /content/drive/MyDrive/deep_classifier_parameters.pkl
